In [ ]:
import pandas as pd
import altair as alt

df1 = pd.read_csv("../data/processed_data.csv")
df1.head() 

,job_title,location,income,skill,Data_base,AWS,ML,WebFM,data_analysis,min_income,max_income,avg_income
0,Pythonエンジニア,"東京都, 神奈川県, 千葉県, 埼玉県, 大阪府, 兵庫県, 京都府, 愛知県, 北海道, ...",504万円〜1080万円,"SQL, Python, Django, AWS, PostgreSQL",1,1,0,1,0,504.0,1080.0,792.0
1,Pythonエンジニア,"東京都, 神奈川県, 千葉県, 埼玉県, 大阪府, 兵庫県, 京都府, 愛知県, 北海道, ...",530万円〜1000万円,"JavaScript, SQL, PHP, Ruby, Java, Python, Swif...",1,1,1,1,1,530.0,1000.0,765.0
2,開発エンジニア,"東京都, 神奈川県, 千葉県, 埼玉県",400万円〜1100万円,"C#, C, C++, Objective-C, JavaScript, VBA, PHP,...",1,1,0,1,0,400.0,1100.0,750.0
3,データエンジニア,東京都,400万円〜600万円,"SQL, Python",1,0,0,0,0,400.0,600.0,500.0
4,Pythonエンジニア,"東京都, 神奈川県, 千葉県, 埼玉県, 大阪府, 愛知県, 福岡県",350万円〜700万円,"JavaScript, PHP, Ruby, Java, Python, Go, CakeP...",1,1,0,1,0,350.0,700.0,525.0


In [3]:
df1.tail()

,job_title,location,income,skill,Data_base,AWS,ML,WebFM,data_analysis,min_income,max_income,avg_income
230,開発エンジニア,宮城県,450万円〜530万円,"Objective-C, JavaScript, PHP, Java, Python, Sw...",0,0,0,0,0,450.0,530.0,490.0
231,SRE,"東京都, 神奈川県, 千葉県, 埼玉県, 大阪府, 兵庫県, 京都府, 愛知県, 北海道, ...",550万円〜1200万円,"Ruby, Python, Ruby on Rails, Django, Node.js, ...",1,1,0,1,0,550.0,1200.0,875.0
232,データアナリスト,"東京都, 神奈川県, 千葉県, 埼玉県, 茨城県, 栃木県, 群馬県, フルリモート",500万円〜1080万円,"SQL, Python, R, Hadoop",1,0,0,0,0,500.0,1080.0,790.0
233,Webエンジニア,京都府,460万円〜650万円,"JavaScript, PHP, Python, jQuery, AWS, Laravel,...",0,1,0,0,0,460.0,650.0,555.0
234,開発エンジニア,大阪府,420万円〜800万円,"C, C++, Java, Python, Go, AWS, Docker, MySQL, ...",1,1,0,0,0,420.0,800.0,610.0


In [4]:

df1.isnull().sum()

job_title        0
location         0
income           1
skill            0
Data_base        0
AWS              0
ML               0
WebFM            0
data_analysis    0
min_income       9
max_income       9
avg_income       9
dtype: int64

#### 課題1「Python関連の主流技術を特定する 」

In [ ]:

skill_counts = df1[["Data_base", "AWS", "ML", "WebFM", "data_analysis"]].sum()

# Altair用データフレームに変換
df_plot = pd.DataFrame({
    "スキル": skill_counts.index,
    "件数": skill_counts.values
})

# 棒グラフ
bars = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X("スキル:N", sort='-y'),
    y=alt.Y("件数:Q")
)

# ラベル（件数表示）
labels = alt.Chart(df_plot).mark_text(
    dy=-6,  # 上にずらす（バーの上に表示）
    fontSize=13,
    fontWeight='bold'
).encode(
    x=alt.X("スキル:N", sort='-y'),
    y="件数:Q",
    text="件数:Q"
)

# 合体して表示
chart = (bars + labels).properties(
    title="スキル出現数の比較",
    width=600,
    height=400
)

chart.show()


alt.LayerChart(...)

#### スキルの組み合わせ

In [ ]:
def get_skill_combination(row):
    return '+'.join([col for col in row.index if row[col] == 1])

#df1["skill_combo"] = df1[["データベース", "AWS", "機械学習", "Webフレームワーク", "データ分析"]].apply(get_skill_combination, axis=1)
df1["skill_combo"] = df1[["Data_base", "AWS", "ML", "WebFM", "data_analysis"]].apply(get_skill_combination, axis=1)
df1 = df1[df1["skill_combo"] != ""]

combo_counts = df1["skill_combo"].value_counts().head(10).reset_index()
combo_counts.columns = ["スキル組み合わせ", "件数"]



bars = alt.Chart(combo_counts).mark_bar().encode(
    x=alt.X("件数:Q"),
    y=alt.Y("スキル組み合わせ:N", sort='-x')
)

labels = alt.Chart(combo_counts).mark_text(
    align='left',
    baseline='middle',
    dx=3,  # 横に少しずらしてバーの右に表示
    fontSize=13
).encode(
    x="件数:Q",
    y=alt.Y("スキル組み合わせ:N", sort='-x'),
    text="件数:Q"
)

chart = (bars + labels).properties(
    title="スキルの組み合わせ出現数（上位10）",
    width=700,
    height=400
)

chart.show()


alt.LayerChart(...)

#### 課題２「Pythonエンジニアの年収レンジを把握する」

In [ ]:
# データを1列に整形（long形式）
df_income = pd.melt(
    df1[["min_income", "max_income", "avg_income"]],
    var_name="収入タイプ",
    value_name="年収"
).dropna()

# Altairのヒストグラム
chart = alt.Chart(df_income).mark_bar(opacity=0.6).encode(
    alt.X("年収:Q", bin=alt.Bin(maxbins=30), title="年収（万円）"),
    alt.Y("count()", title="件数"),
    alt.Color("収入タイプ:N", title="タイプ")
).properties(
    title="min_income / max_income / avg_income の分布",
    width=700,
    height=400
)

chart.show()


alt.Chart(...)

#### 課題２の展開：スキル組み合わせ別の年収レンジ

In [20]:
# データのロング形式変換（melt）
df_melt = df1[df1["skill_combo"].isin(top_combos)].melt(
    id_vars="skill_combo",
    value_vars=["min_income", "avg_income", "max_income"],
    var_name="income_type",
    value_name="income_value"
)

# Altairで箱ひげ図描画
box_chart = alt.Chart(df_melt).mark_boxplot(extent='min-max').encode(
    y=alt.Y('skill_combo:N', sort='-x', title='スキル組み合わせ'),
    x=alt.X('income_value:Q', title='年収（万円）'),
    color=alt.Color('income_type:N', title="収入タイプ",
                    scale=alt.Scale(domain=["min_income", "avg_income", "max_income"],
                                    range=["#FFA07A", "#87CEFA", "#90EE90"]))
).properties(
    width=800,
    height=500,
    title="スキル組み合わせ別の年収（min / avg / max）分布（箱ひげ図）"
)

box_chart.show()


alt.Chart(...)

#### 課題3「地方別の需要傾向を特定する」

In [22]:
region_map = {
    "北海道地方": ["北海道"],
    "東北地方": ["青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県"],
    "関東地方": ["茨城県", "栃木県", "群馬県", "埼玉県", "千葉県", "神奈川県"],
    "東京都": ["東京都"], # 特別扱い
    "中部地方": ["新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県", "岐阜県", "静岡県", "愛知県"],
    "近畿地方": ["三重県", "滋賀県", "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県"],
    "中国地方": ["鳥取県", "島根県", "岡山県", "広島県", "山口県"],
    "四国地方": ["徳島県", "香川県", "愛媛県", "高知県"],
    "九州地方": ["福岡県", "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", "鹿児島県", "沖縄県"]
}

# 都道府県から地方名に変換する辞書
pref_to_region = {}
for region, prefs in region_map.items():
    for pref in prefs:
        pref_to_region[pref] = region

print(pref_to_region)

{'北海道': '北海道地方', '青森県': '東北地方', '岩手県': '東北地方', '宮城県': '東北地方', '秋田県': '東北地方', '山形県': '東北地方', '福島県': '東北地方', '茨城県': '関東地方', '栃木県': '関東地方', '群馬県': '関東地方', '埼玉県': '関東地方', '千葉県': '関東地方', '神奈川県': '関東地方', '東京都': '東京都', '新潟県': '中部地方', '富山県': '中部地方', '石川県': '中部地方', '福井県': '中部地方', '山梨県': '中部地方', '長野県': '中部地方', '岐阜県': '中部地方', '静岡県': '中部地方', '愛知県': '中部地方', '三重県': '近畿地方', '滋賀県': '近畿地方', '京都府': '近畿地方', '大阪府': '近畿地方', '兵庫県': '近畿地方', '奈良県': '近畿地方', '和歌山県': '近畿地方', '鳥取県': '中国地方', '島根県': '中国地方', '岡山県': '中国地方', '広島県': '中国地方', '山口県': '中国地方', '徳島県': '四国地方', '香川県': '四国地方', '愛媛県': '四国地方', '高知県': '四国地方', '福岡県': '九州地方', '佐賀県': '九州地方', '長崎県': '九州地方', '熊本県': '九州地方', '大分県': '九州地方', '宮崎県': '九州地方', '鹿児島県': '九州地方', '沖縄県': '九州地方'}


In [23]:
# location列を分解し、地域ごとの集計を作成
from collections import Counter

region_counter = Counter()

for locs in df1["location"]:
    for pref in [p.strip() for p in locs.split(",")]:
        region = pref_to_region.get(pref)
        if region:
            region_counter[region] += 1

# 結果をデータフレームに変換
region_df = pd.DataFrame.from_dict(region_counter, orient='index', columns=['count']).reset_index()
region_df.columns = ['region', 'count']

In [ ]:
# 棒グラフ
bars = alt.Chart(region_df).mark_bar().encode(
    x=alt.X('region:N', sort='-y', title='地方区分'),
    y=alt.Y('count:Q', title='出現回数'),
    color='region:N'
).properties(
    width=600,
    height=400,
    title='地方別出現回数'
)

# ラベル（件数表示）
labels = alt.Chart(region_df).mark_text(
    dy=-6,
    fontSize=13,
    fontWeight='bold'
).encode(
    x=alt.X('region:N', sort='-y'),
    y='count:Q',
    text='count:Q'
)

# 合体して表示
chart = (bars + labels).properties(
    title="地方別出現回数",
    width=600,
    height=400
)

chart


alt.LayerChart(...)